In [1]:
%load_ext autoreload
%autoreload 2
import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer
from tqdm import tqdm_pandas
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
warnings.filterwarnings('ignore')

/home/charles/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/charles/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
keppels_files = glob2.glob('files/Keppel/*')
prudential_files = glob2.glob('files/Prudential/*')

In [3]:
files = pd.DataFrame(keppels_files, columns=['file_path'])
#files.index = files['file_path']
files['sentiment'] = np.nan
#files.loc['files/Keppel/02224.doc'].sentiment
files[0:10].head()

,file_path,sentiment
0,files/Keppel/KLL_AR2002a.pdf,NaN
1,files/Keppel/02224.doc,NaN
2,files/Keppel/AB3Q2012.pdf,NaN
3,files/Keppel/Lewin_130908_Sup.pdf,NaN
4,files/Keppel/AB2-2017.pdf,NaN


In [4]:
def find_sentiment(path):
    text = extract_text.apply(path)
    important_sents = text_summarization.apply(text)
    #print(len(important_sents))
    importance = np.mean(sentiment_analyzer.apply(important_sents))
    return np.mean(importance)

In [5]:
tqdm().pandas(desc="Progress:")
files[1:6]['sentiment'] = files[1:6].progress_apply(lambda row: find_sentiment(row.file_path), axis=1)

A Jupyter Widget

A Jupyter Widget

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead



In [9]:
files.head()

,file_path,sentiment
0,files/Keppel/KLL_AR2002a.pdf,NaN
1,files/Keppel/02224.doc,NaN
2,files/Keppel/AB3Q2012.pdf,0.641253
3,files/Keppel/Lewin_130908_Sup.pdf,NaN
4,files/Keppel/AB2-2017.pdf,0.701296


In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11461191964243711662]

In [8]:
sentiment_analyzer.apply(['I am ugly', 'success'])

array([[0.22300728],
       [0.7965621 ]], dtype=float32)